In [18]:
from NAPC import NAPC, conf
import mysql.connector
import pandas as pd

# MySQL connection parameters
mysql_config = {
    'user': 'handy',
    'password': 'handy',
    'host': 'localhost',
    'database': 'framework_2'
}

# update with your IBM Token
token = "put_your_ibm_token_here"

# Running to the noisy simulator

## Example of code of length N=16 (n=3) for logical $|+\rangle$

In [2]:
# initialize the object using the provided token
q = NAPC(runs=conf.runs, token=token)

Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information of ibm_sherbrooke...


In [2]:
# initialize the object using the provided token
q = NAPC(runs=conf.runs, token=token)

# command to run the update TriQ's reliability matrix for the noise-aware routing
q.update_hardware_configs()

# then specify which polar codes we want to run, for example polar codes for logical + with length N=16 (n=3)
file_path="./QEC/polar_code/n3/x"

qasm_files = q.get_qasm_files_from_path(file_path)

# then decide which compilation to be run Qiskit-3 or S-TriQ (triq_lcd_sabre)
compilations=["qiskit_3", "triq_lcd_sabre"]

# this is the setup to enable mid measurement for TriQ
triq_measurement_type="polar_meas"

# choosing the noise level
noise_levels = [0.1, 1.0]

# setup the number of shots
shots = 1000

# then we setup the backend and run the simulator
q.set_backend(program_type="sampler", shots=shots)
q.run_simulator("sampler", qasm_files, compilations, noise_levels, shots)

Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information of ibm_sherbrooke...
./QEC/polar_code/n3/x
Start running the simulator...
qiskit_3 0.1
qiskit_3 1.0
triq_lcd_sabre 0.1
triq_lcd_sabre 1.0
Total send to local simulator : 2
Running to Local Simulator with batch id: 1 ... 
Sent!
Running to Local Simulator with batch id: 2 ... 
Sent!


In [5]:
# get the result and calculate the metric
q.get_qiskit_result("simulator")

Pending jobs:  0
Executed jobs : 2
Getting qasm for : 1 simulator
get metrics: n = 3 , lstate = X
number of discarded states (invalid measurement results) =  2.0
 number of accepted states   (valid measurement results) =  998.0
        number of logical errors on the accepted states  =  4
polar_all_meas_n3_x 0.1 qiskit_3 998.0 4 0 0.9959919839679359
get metrics: n = 3 , lstate = X
number of discarded states (invalid measurement results) =  64.0
 number of accepted states   (valid measurement results) =  936.0
        number of logical errors on the accepted states  =  4
polar_all_meas_n3_x 1.0 qiskit_3 936.0 4 0 0.9957264957264957
Getting qasm for : 2 simulator
get metrics: n = 3 , lstate = X
number of discarded states (invalid measurement results) =  4.0
 number of accepted states   (valid measurement results) =  996.0
        number of logical errors on the accepted states  =  4
polar_all_meas_n3_x 0.1 qiskit_3 996.0 4 0 0.9959839357429718
get metrics: n = 3 , lstate = X
number of di

**After this, you can check the result in the database**

# Running to the real backend

## Example of code of length N=32 (n=4) for logical $|0\rangle$

In [6]:
# initialize the object using the provided token
q = NAPC(runs=conf.runs, token=token)

# command to run the update TriQ's reliability matrix for the noise-aware routing
q.update_hardware_configs()

# then specify which polar codes we want to run, for example polar codes for logical + with length N=16 (n=3)
file_path="./QEC/polar_code/n4/z"

qasm_files = q.get_qasm_files_from_path(file_path)

# then decide which compilation to be run Qiskit-3 or S-TriQ (triq_lcd_sabre)
compilations=["qiskit_3", "triq_lcd_sabre"]

# this is the setup to enable mid measurement for TriQ
triq_measurement_type="polar_meas"

# setup the number of shots
shots = 1000

# then we setup the backend and run the simulator
q.set_backend(program_type="sampler", shots=shots)



Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information of ibm_sherbrooke...
./QEC/polar_code/n4/z


**The only different with sending to the simulator is this part**

In [7]:
q.send_to_real_backend("sampler", qasm_files, compilations, shots=shots)

Compiling circuit: polar_all_meas_n4 for compilation: qiskit_3
Compiling circuit: polar_all_meas_n4 for compilation: triq_lcd_sabre
Total send to real backend : 1
Total no of circuits : 2
Sending to ibm_sherbrooke with batch id: 3 ... 
Sent!


For other code of lengths simulation, see the file "**run_simulation.py**" and "**run_to_real_backend.py**"

# Accessing the result in the database

In [15]:
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

sql = """
SELECT header_id, circuit_name, compilation_name, noise_level, 
polar_count_accept, polar_count_logerror, polar_count_undecided, success_rate_polar, total_two_qubit_gate,
initial_mapping
FROM result WHERE header_id = 2;
""".format(1, "ibm_sherbrooke")

 # insert to circuit
cursor.execute(sql)

results = cursor.fetchall()

cursor.close()
conn.close()

In [19]:
header_id = []
circuit_name = []
compilation_name = []
noise_level = []
count_accept = []
count_logerror = []
count_undecided = []
success_polar = []
total_gate_cx = []
initial_mappings = []


for res in results:
    _header_id, _circuit_name, _compilation_name , _noise_level, _count_accept, _count_logerror, \
    _count_undecided, _success_polar, _total_gate_cx, _initial_mappings = res

    header_id.append(_header_id)
    circuit_name.append(_circuit_name)
    compilation_name.append(_compilation_name)
    noise_level.append(float(_noise_level))
    count_accept.append(_count_accept)
    count_logerror.append(_count_logerror)
    count_undecided.append(_count_undecided)
    success_polar.append(1-float(_success_polar))
    total_gate_cx.append(float(_total_gate_cx))
    initial_mappings.append(_initial_mappings)

df = pd.DataFrame({
    'header_id': header_id,
    'circuit_name': circuit_name,
    'compilation_name': compilation_name,
    'noise_level': noise_level,
    'count_accept': count_accept,
    'count_logerror': count_logerror, 
    'count_undecided': count_undecided, 
    'success_polar': success_polar,
    'total_gate_cx': total_gate_cx,
    'noise_level': noise_level,
    'initial_mappings': initial_mappings
    })


In [28]:
print(df.head().to_markdown(tablefmt="grid"))

+----+-------------+---------------------+--------------------+---------------+----------------+------------------+-------------------+-----------------+-----------------+------------------------------+
|    |   header_id | circuit_name        | compilation_name   |   noise_level |   count_accept |   count_logerror |   count_undecided |   success_polar |   total_gate_cx | initial_mappings             |
+====+=============+=====================+====================+===============+================+==================+===================+=================+=================+==============================+
|  0 |           2 | polar_all_meas_n3_x | qiskit_3           |           0.1 |            996 |                4 |                 0 |        0.004016 |              11 | 123, 121, 122, 120, 118, 119 |
+----+-------------+---------------------+--------------------+---------------+----------------+------------------+-------------------+-----------------+-----------------+-----------------